# Dec 2022
# Clean up DIA and MED tables

In [1]:
!pip install fastparquet

  Using cached fastparquet-2022.12.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.6 MB)
  Using cached cramjam-2.6.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.0 MB)


In [2]:
import pandas as pd
import numpy as np
import os as os

pd.set_option('display.max_rows', 500)

import seaborn as sns
import matplotlib as plt

datadir = '/challenge/seeing-through-the-fog/data/train_data'

In [3]:
df_dia = pd.read_parquet(datadir + "/" + "diagnoses.parquet")
df_obs = pd.read_parquet(datadir + "/" + "observations.parquet")
df_med = pd.read_parquet(datadir + "/" + "medication.parquet")
df_lab = pd.read_parquet(datadir + "/" + "labs.parquet")
df_imm = pd.read_parquet(datadir + "/" + "immunization.parquet")
df_dem = pd.read_parquet(datadir + "/" + "demo.parquet")
df_tar = pd.read_parquet(datadir + "/" + "target.parquet")

# df_dia

## df_dia consolidate diagnosis_cd, only keep meaningful codes

In [4]:
df_dia.head()

,patientid,days_to_covid_diag,diagnosis_cd,diagnosis_cd_type
0,RAADC3-395646,49,M545,ICD10
1,RAADC3-395646,49,R109,ICD10
2,RAADC3-242960,29,N898,ICD10
3,RAADC3-242960,29,N898,ICD10
4,RAADC3-242960,29,N760,ICD10


In [5]:
df_dia.shape

(8344797, 4)

In [6]:
# deduplicate
df_dia_dedup = df_dia.drop_duplicates()
df_dia_dedup.shape

(4599783, 4)

In [7]:
df_dia_dedup['diagnosis_cd_type'].unique()

array(['ICD10', 'ICD9', 'UNKNOWN', 'SNOMED', 'OTHER'], dtype=object)

In [8]:
df_dia_dedup['diagnosis_cd'].unique()

array(['M545', 'R109', 'N898', ..., 'S21009A', 'S80822S', 'W5519XA'],
      dtype=object)

In [9]:
len(df_dia_dedup['diagnosis_cd'].unique())

34077

### Combine "diagnosis_cd" and "diagnosis_cd_type" into one column, count frequency, pick top 200

In [10]:
df_dia_dedup['diag_cd_and_type'] = df_dia_dedup['diagnosis_cd'] + "_" + df_dia_dedup['diagnosis_cd_type']

/tmp/ipykernel_1522/3180494961.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dia_dedup['diag_cd_and_type'] = df_dia_dedup['diagnosis_cd'] + "_" + df_dia_dedup['diagnosis_cd_type']


In [11]:
df_dia_dedup.head()

,patientid,days_to_covid_diag,diagnosis_cd,diagnosis_cd_type,diag_cd_and_type
0,RAADC3-395646,49,M545,ICD10,M545_ICD10
1,RAADC3-395646,49,R109,ICD10,R109_ICD10
2,RAADC3-242960,29,N898,ICD10,N898_ICD10
4,RAADC3-242960,29,N760,ICD10,N760_ICD10
7,RAADC3-242960,29,B9689,ICD10,B9689_ICD10


In [12]:
df_dia_dedup_top_cd_and_types = df_dia_dedup.groupby('diag_cd_and_type')[['patientid']].count().\
reset_index().sort_values(by='patientid', ascending=False)

In [13]:
df_dia_dedup_top_cd_and_types.head(150)

,diag_cd_and_type,patientid
17623,I10_ICD10,121207
33249,Z20828_ICD10,80522
14676,E785_ICD10,53261
33738,Z79899_ICD10,46180
14427,E119_ICD10,44168
33248,Z20822_ICD10,40449
18990,K219_ICD10,37436
24631,R05_ICD10,34378
33266,Z23_ICD10,32737
17672,I2510_ICD10,32075


In [14]:
df_dia_dedup_top_cd_and_types.shape

(34141, 2)

In [15]:
df_dia_codes_top_200 = df_dia_dedup_top_cd_and_types.head(200)['diag_cd_and_type'].tolist()

In [16]:
df_dia_codes_top_200

['I10_ICD10',
 'Z20828_ICD10',
 'E785_ICD10',
 'Z79899_ICD10',
 'E119_ICD10',
 'Z20822_ICD10',
 'K219_ICD10',
 'R05_ICD10',
 'Z23_ICD10',
 'I2510_ICD10',
 'Z0000_ICD10',
 'F419_ICD10',
 'E039_ICD10',
 'Z1159_ICD10',
 'Z87891_ICD10',
 'Z794_ICD10',
 'G4733_ICD10',
 'UNK_UNKNOWN',
 'E6601_ICD10',
 'G8929_ICD10',
 'E669_ICD10',
 'Z7901_ICD10',
 'F329_ICD10',
 'J449_ICD10',
 'R509_ICD10',
 'E782_ICD10',
 'E1165_ICD10',
 'R0602_ICD10',
 'E559_ICD10',
 'D649_ICD10',
 'M545_ICD10',
 'N179_ICD10',
 'F411_ICD10',
 '7862_ICD9',
 'I4891_ICD10',
 'N186_ICD10',
 'I480_ICD10',
 'E1122_ICD10',
 'R5383_ICD10',
 'Z1231_ICD10',
 'Z01818_ICD10',
 'Z7982_ICD10',
 'J069_ICD10',
 'N390_ICD10',
 'J45909_ICD10',
 'R079_ICD10',
 'Z03818_ICD10',
 'F17210_ICD10',
 'R6889_ICD10',
 'J189_ICD10',
 'J029_ICD10',
 'E7800_ICD10',
 'Z98890_ICD10',
 '2722_ICD9',
 'R109_ICD10',
 'Z992_ICD10',
 'E876_ICD10',
 'J9601_ICD10',
 'R519_ICD10',
 'M542_ICD10',
 'D631_ICD10',
 'I509_ICD10',
 'I129_ICD10',
 'NA_UNKNOWN',
 'R69_ICD

In [17]:
df_dia_dedup_200_commonly_seen_codes = df_dia_dedup[df_dia_dedup['diag_cd_and_type'].isin(df_dia_codes_top_200)]

In [18]:
df_dia_dedup_200_commonly_seen_codes.shape

(2218960, 5)

### Total of 34141 unique codes for 4599783 unique records. 
### If pick the top 200 codes only, cover 2218960 records (48% of information retained)

In [19]:
df_dia_dedup.shape

(4599783, 5)

In [20]:
## Look into a few of the top codes, explore explanable codes

### 1. I10_ICD10: Hypertensive diseases
### 2. Z20828_ICD10: Contact with and (suspected) exposure to communicable diseases
### 3. E785_ICD10: Hyperlipidemia
### 4. Z79899_ICD10: Other long term (current) drug therapy
### 5. E119_ICD10: Type 2 diabetes mellitus without complications
### 6. Z20822_ICD10: Contact with and (suspected) exposure to COVID-19
### 7. K219_ICD10: Gastro-esophageal reflux disease without esophagitis
### 8. R05_ICD10: Cough

### ICD-10 codes more recent and with more records for this table
### ICD-10 can be split by letter to certain categories
### ICD-9 outdated, not as abundantly recorded
### ICD-9 has number system

In [22]:
print(df_dia_dedup[df_dia_dedup['diagnosis_cd_type']=='ICD10'].shape)
print(df_dia_dedup[df_dia_dedup['diagnosis_cd_type']=='ICD9'].shape)
print(df_dia_dedup[df_dia_dedup['diagnosis_cd_type']=='SNOMED'].shape)

(4204725, 5)
(178840, 5)
(176476, 5)


In [21]:
### understand the codes
'''
2023 ICD-10-CM Codes
Categories

A00-B99  Certain infectious and parasitic diseases
C00-D49  Neoplasms
D50-D89  Diseases of the blood and blood-forming organs and certain disorders involving the immune mechanism
E00-E89  Endocrine, nutritional and metabolic diseases
F01-F99  Mental, Behavioral and Neurodevelopmental disorders
G00-G99  Diseases of the nervous system
H00-H59  Diseases of the eye and adnexa
H60-H95  Diseases of the ear and mastoid process
I00-I99  Diseases of the circulatory system
J00-J99  Diseases of the respiratory system
K00-K95  Diseases of the digestive system
L00-L99  Diseases of the skin and subcutaneous tissue
M00-M99  Diseases of the musculoskeletal system and connective tissue
N00-N99  Diseases of the genitourinary system
O00-O9A  Pregnancy, childbirth and the puerperium
P00-P96  Certain conditions originating in the perinatal period
Q00-Q99  Congenital malformations, deformations and chromosomal abnormalities
R00-R99  Symptoms, signs and abnormal clinical and laboratory findings, not elsewhere classified
S00-T88  Injury, poisoning and certain other consequences of external causes
U00-U85  Codes for special purposes
V00-Y99  External causes of morbidity
Z00-Z99  Factors influencing health status and contact with health services



2015 ICD-9-CM Diagnosis Codes >
001-139  Infectious And Parasitic Diseases
140-239  Neoplasms
240-279  Endocrine, Nutritional And Metabolic Diseases, And Immunity Disorders
280-289  Diseases Of The Blood And Blood-Forming Organs
290-319  Mental Disorders
320-389  Diseases Of The Nervous System And Sense Organs
390-459  Diseases Of The Circulatory System
460-519  Diseases Of The Respiratory System
520-579  Diseases Of The Digestive System
580-629  Diseases Of The Genitourinary System
630-679  Complications Of Pregnancy, Childbirth, And The Puerperium
680-709  Diseases Of The Skin And Subcutaneous Tissue
710-739  Diseases Of The Musculoskeletal System And Connective Tissue
740-759  Congenital Anomalies
760-779  Certain Conditions Originating In The Perinatal Period
780-799  Symptoms, Signs, And Ill-Defined Conditions
800-999  Injury And Poisoning
V01-V91  Supplementary Classification Of Factors Influencing Health Status And Contact With Health Services
E000-E999  Supplementary Classification Of External Causes Of Injury And Poisoning
'''

'\n2023 ICD-10-CM Codes\nCategories\n\nA00-B99  Certain infectious and parasitic diseases\nC00-D49  Neoplasms\nD50-D89  Diseases of the blood and blood-forming organs and certain disorders involving the immune mechanism\nE00-E89  Endocrine, nutritional and metabolic diseases\nF01-F99  Mental, Behavioral and Neurodevelopmental disorders\nG00-G99  Diseases of the nervous system\nH00-H59  Diseases of the eye and adnexa\nH60-H95  Diseases of the ear and mastoid process\nI00-I99  Diseases of the circulatory system\nJ00-J99  Diseases of the respiratory system\nK00-K95  Diseases of the digestive system\nL00-L99  Diseases of the skin and subcutaneous tissue\nM00-M99  Diseases of the musculoskeletal system and connective tissue\nN00-N99  Diseases of the genitourinary system\nO00-O9A  Pregnancy, childbirth and the puerperium\nP00-P96  Certain conditions originating in the perinatal period\nQ00-Q99  Congenital malformations, deformations and chromosomal abnormalities\nR00-R99  Symptoms, signs and

In [23]:
df_dia_dedup[df_dia_dedup['diagnosis_cd_type']=='ICD9']

,patientid,days_to_covid_diag,diagnosis_cd,diagnosis_cd_type,diag_cd_and_type
32,RAADC3-468258,16,4928,ICD9,4928_ICD9
97,RAADC3-467760,6,3384,ICD9,3384_ICD9
98,RAADC3-467760,39,3384,ICD9,3384_ICD9
107,RAADC3-467760,61,3384,ICD9,3384_ICD9
158,RAADC3-023996,54,5950,ICD9,5950_ICD9
...,...,...,...,...,...
8338189,RAADC3-041674,56,V8531,ICD9,V8531_ICD9
8338351,RAADC3-593972,23,V860,ICD9,V860_ICD9
8338421,RAADC3-593972,57,V860,ICD9,V860_ICD9
8338457,RAADC3-582076,47,V8801,ICD9,V8801_ICD9


In [24]:
df_dia_ICD10 = df_dia_dedup[df_dia_dedup['diagnosis_cd_type']=='ICD10']

In [25]:
df_dia_ICD10.head()

,patientid,days_to_covid_diag,diagnosis_cd,diagnosis_cd_type,diag_cd_and_type
0,RAADC3-395646,49,M545,ICD10,M545_ICD10
1,RAADC3-395646,49,R109,ICD10,R109_ICD10
2,RAADC3-242960,29,N898,ICD10,N898_ICD10
4,RAADC3-242960,29,N760,ICD10,N760_ICD10
7,RAADC3-242960,29,B9689,ICD10,B9689_ICD10


In [26]:
## Drop numbers in the ICD10, use letter as the category

#df_dia_ICD10['dia_icd10_short'] = df_dia_ICD10['diagnosis_cd'].replace('\d+', '', regex=True)
df_dia_ICD10['dia_icd10_short'] = df_dia_ICD10['diagnosis_cd'].astype(str).str[0]

/tmp/ipykernel_1522/3086762324.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dia_ICD10['dia_icd10_short'] = df_dia_ICD10['diagnosis_cd'].astype(str).str[0]


In [27]:
df_dia_ICD10.head()

,patientid,days_to_covid_diag,diagnosis_cd,diagnosis_cd_type,diag_cd_and_type,dia_icd10_short
0,RAADC3-395646,49,M545,ICD10,M545_ICD10,M
1,RAADC3-395646,49,R109,ICD10,R109_ICD10,R
2,RAADC3-242960,29,N898,ICD10,N898_ICD10,N
4,RAADC3-242960,29,N760,ICD10,N760_ICD10,N
7,RAADC3-242960,29,B9689,ICD10,B9689_ICD10,B


In [28]:
df_dia_ICD10_short = df_dia_ICD10[['patientid', 'days_to_covid_diag', 'dia_icd10_short']].drop_duplicates()

In [29]:
df_dia_ICD10_short.shape

(2482298, 3)

In [30]:
df_dia_ICD10_short.head()

,patientid,days_to_covid_diag,dia_icd10_short
0,RAADC3-395646,49,M
1,RAADC3-395646,49,R
2,RAADC3-242960,29,N
7,RAADC3-242960,29,B
8,RAADC3-542620,48,R


### If not treat the ICD code as time series, then record the YES/NO status of the categories of ICD

In [31]:
df_dia_ICD10_short_by_patient = df_dia_ICD10_short[['patientid', 'dia_icd10_short']].drop_duplicates()
df_dia_ICD10_short_by_patient['ICD10_status'] = 1

In [32]:
df_dia_ICD10_short_by_patient.shape

(997613, 3)

In [33]:
## Peek pivoted table

df_dia_ICD10_short_by_patient.pivot(index=["patientid"], \
                         columns=["dia_icd10_short"], values=['ICD10_status'])

ICD10_status                                             ...  \
dia_icd10_short            A   B    C    D    E    F    G    H    I   J  ...   
patientid                                                                ...   
RAADC3-000002            NaN NaN  NaN  1.0  1.0  1.0  1.0  NaN  1.0 NaN  ...   
RAADC3-000004            NaN NaN  NaN  NaN  1.0  NaN  NaN  NaN  1.0 NaN  ...   
RAADC3-000005            NaN NaN  NaN  NaN  NaN  NaN  1.0  NaN  NaN NaN  ...   
RAADC3-000006            NaN NaN  NaN  1.0  NaN  NaN  NaN  NaN  1.0 NaN  ...   
RAADC3-000008            NaN NaN  NaN  NaN  1.0  1.0  NaN  1.0  1.0 NaN  ...   
...                      ...  ..  ...  ...  ...  ...  ...  ...  ...  ..  ...   
RAADC3-727935            NaN NaN  NaN  1.0  1.0  NaN  NaN  NaN  NaN NaN  ...   
RAADC3-727950            NaN NaN  NaN  NaN  NaN  1.0  NaN  NaN  NaN NaN  ...   
RAADC3-727953            NaN NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN NaN  ...   
RAADC3-727954            NaN NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN NaN  ...   
RAADC3-727960            NaN NaN  1.0  1.0  NaN  NaN  NaN  NaN  NaN NaN  ...   

                                                            
dia_icd10_short   Q    R    S   T   U   V   W   X   Y    Z  
patientid                                                   
RAADC3-000002   NaN  1.0  NaN NaN NaN NaN NaN NaN NaN  1.0  
RAADC3-000004   NaN  1.0  NaN NaN NaN NaN NaN NaN NaN  NaN  
RAADC3-000005   NaN  1.0  1.0 NaN NaN NaN NaN NaN NaN  NaN  
RAADC3-000006   NaN  1.0  NaN NaN NaN NaN NaN NaN NaN  1.0  
RAADC3-000008   NaN  NaN  NaN NaN NaN NaN NaN NaN NaN  1.0  
...              ..  ...  ...  ..  ..  ..  ..  ..  ..  ...  
RAADC3-727935   NaN  1.0  1.0 NaN NaN NaN NaN NaN NaN  1.0  
RAADC3-727950   NaN  NaN  NaN NaN NaN NaN NaN NaN NaN  1.0  
RAADC3-727953   NaN  1.0  NaN NaN NaN NaN NaN NaN NaN  1.0  
RAADC3-727954   NaN  1.0  NaN NaN NaN NaN NaN NaN NaN  NaN  
RAADC3-727960   NaN  NaN  NaN NaN NaN NaN NaN NaN NaN  1.0  

[261901 rows x 26 columns]

In [34]:
df_dia_ICD10_pivot_table = df_dia_ICD10_short_by_patient.pivot(index=["patientid"], \
                         columns=["dia_icd10_short"], values=['ICD10_status']).fillna(0).reset_index()

In [35]:
df_dia_ICD10_pivot_table

patientid ICD10_status                                \
dia_icd10_short                           A    B    C    D    E    F    G   
0                RAADC3-000002          0.0  0.0  0.0  1.0  1.0  1.0  1.0   
1                RAADC3-000004          0.0  0.0  0.0  0.0  1.0  0.0  0.0   
2                RAADC3-000005          0.0  0.0  0.0  0.0  0.0  0.0  1.0   
3                RAADC3-000006          0.0  0.0  0.0  1.0  0.0  0.0  0.0   
4                RAADC3-000008          0.0  0.0  0.0  0.0  1.0  1.0  0.0   
...                        ...          ...  ...  ...  ...  ...  ...  ...   
261896           RAADC3-727935          0.0  0.0  0.0  1.0  1.0  0.0  0.0   
261897           RAADC3-727950          0.0  0.0  0.0  0.0  0.0  1.0  0.0   
261898           RAADC3-727953          0.0  0.0  0.0  0.0  0.0  0.0  0.0   
261899           RAADC3-727954          0.0  0.0  0.0  0.0  0.0  0.0  0.0   
261900           RAADC3-727960          0.0  0.0  1.0  1.0  0.0  0.0  0.0   

                           ...                                               \
dia_icd10_short    H    I  ...    Q    R    S    T    U    V    W    X    Y   
0                0.0  1.0  ...  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1                0.0  1.0  ...  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
2                0.0  0.0  ...  0.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0   
3                0.0  1.0  ...  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
4                1.0  1.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
...              ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
261896           0.0  0.0  ...  0.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0   
261897           0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
261898           0.0  0.0  ...  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
261899           0.0  0.0  ...  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
261900           0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

                      
dia_icd10_short    Z  
0                1.0  
1                0.0  
2                0.0  
3                1.0  
4                1.0  
...              ...  
261896           1.0  
261897           1.0  
261898           1.0  
261899           0.0  
261900           1.0  

[261901 rows x 27 columns]

In [36]:
df_dia_ICD10_pivot_table.columns

MultiIndex([(   'patientid',  ''),
            ('ICD10_status', 'A'),
            ('ICD10_status', 'B'),
            ('ICD10_status', 'C'),
            ('ICD10_status', 'D'),
            ('ICD10_status', 'E'),
            ('ICD10_status', 'F'),
            ('ICD10_status', 'G'),
            ('ICD10_status', 'H'),
            ('ICD10_status', 'I'),
            ('ICD10_status', 'J'),
            ('ICD10_status', 'K'),
            ('ICD10_status', 'L'),
            ('ICD10_status', 'M'),
            ('ICD10_status', 'N'),
            ('ICD10_status', 'O'),
            ('ICD10_status', 'P'),
            ('ICD10_status', 'Q'),
            ('ICD10_status', 'R'),
            ('ICD10_status', 'S'),
            ('ICD10_status', 'T'),
            ('ICD10_status', 'U'),
            ('ICD10_status', 'V'),
            ('ICD10_status', 'W'),
            ('ICD10_status', 'X'),
            ('ICD10_status', 'Y'),
            ('ICD10_status', 'Z')],
           names=[None, 'dia_icd10_short'])

In [37]:
new_column_names = []
for (row1, row2) in df_dia_ICD10_pivot_table.columns:
    column_name = str(row1) + "_" + str(row2)
    print(column_name)
    new_column_names.append(column_name)

patientid_
ICD10_status_A
ICD10_status_B
ICD10_status_C
ICD10_status_D
ICD10_status_E
ICD10_status_F
ICD10_status_G
ICD10_status_H
ICD10_status_I
ICD10_status_J
ICD10_status_K
ICD10_status_L
ICD10_status_M
ICD10_status_N
ICD10_status_O
ICD10_status_P
ICD10_status_Q
ICD10_status_R
ICD10_status_S
ICD10_status_T
ICD10_status_U
ICD10_status_V
ICD10_status_W
ICD10_status_X
ICD10_status_Y
ICD10_status_Z


In [38]:
df_dia_ICD10_pivot_table.columns = new_column_names

In [39]:
df_dia_ICD10_pivot_table.head()

,patientid_,ICD10_status_A,ICD10_status_B,ICD10_status_C,ICD10_status_D,ICD10_status_E,ICD10_status_F,ICD10_status_G,ICD10_status_H,ICD10_status_I,...,ICD10_status_Q,ICD10_status_R,ICD10_status_S,ICD10_status_T,ICD10_status_U,ICD10_status_V,ICD10_status_W,ICD10_status_X,ICD10_status_Y,ICD10_status_Z
0,RAADC3-000002,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,RAADC3-000004,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,RAADC3-000005,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,RAADC3-000006,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,RAADC3-000008,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [40]:
df_dia_ICD10_pivot_table.shape

(261901, 27)

In [41]:
df_dia_ICD10_pivot_table.describe()

,ICD10_status_A,ICD10_status_B,ICD10_status_C,ICD10_status_D,ICD10_status_E,ICD10_status_F,ICD10_status_G,ICD10_status_H,ICD10_status_I,ICD10_status_J,...,ICD10_status_Q,ICD10_status_R,ICD10_status_S,ICD10_status_T,ICD10_status_U,ICD10_status_V,ICD10_status_W,ICD10_status_X,ICD10_status_Y,ICD10_status_Z
count,261901.000000,261901.000000,261901.000000,261901.000000,261901.000000,261901.000000,261901.000000,261901.000000,261901.000000,261901.000000,...,261901.000000,261901.000000,261901.000000,261901.000000,261901.000000,261901.000000,261901.000000,261901.000000,261901.000000,261901.000000
mean,0.019572,0.087354,0.038893,0.115838,0.356879,0.211546,0.186372,0.075578,0.295058,0.251374,...,0.010752,0.529223,0.064188,0.035193,0.001207,0.004173,0.018961,0.005235,0.011111,0.745217
std,0.138525,0.282353,0.193339,0.320031,0.479080,0.408405,0.389407,0.264322,0.456070,0.433804,...,0.103134,0.499146,0.245089,0.184267,0.034715,0.064467,0.136389,0.072162,0.104822,0.435740
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [42]:
## Count how many ICD_10_status per category, use these columns as features for each patient
df_dia_ICD10_pivot_table.sum(axis = 0, skipna = True)

patientid_        RAADC3-000002RAADC3-000004RAADC3-000005RAADC3-...
ICD10_status_A                                               5126.0
ICD10_status_B                                              22878.0
ICD10_status_C                                              10186.0
ICD10_status_D                                              30338.0
ICD10_status_E                                              93467.0
ICD10_status_F                                              55404.0
ICD10_status_G                                              48811.0
ICD10_status_H                                              19794.0
ICD10_status_I                                              77276.0
ICD10_status_J                                              65835.0
ICD10_status_K                                              43704.0
ICD10_status_L                                              23451.0
ICD10_status_M                                              72450.0
ICD10_status_N                                  

In [43]:
df_dia_ICD10_pivot_table.to_csv("/home/huangz36/dia_table_pivot_by_ICD10_code.txt", sep='\t', \
                                index=False, header=True)

# df_MED table

In [44]:
df_med.head()

,patientid,days_to_covid_diag,drug_name,ndc
0,RAADC3-504477,21,Mupirocin,68462018022
1,RAADC3-241668,15,Glucose Meter Test Control Strips,53885024450
2,RAADC3-125094,55,Ergocalciferol,69452015120
3,RAADC3-349386,15,Vitamin B Complex and Vitamin C,60258016001
4,RAADC3-556667,21,Tiagabine Hydrochloride,00093503156


In [45]:
df_med.shape

(6021819, 4)

In [46]:
df_med.drop_duplicates().shape

(4473741, 4)

In [47]:
df_med_dedup = df_med.drop_duplicates()

### Plan:
### See if time series necessary, can the "days_to_covid_diag" be converted to "RESPONSE_OR_NOT", "EARLY_LATE_RESPONSE"?

### Any drug <7 days, 7-14 days, >14 days
### Symptom relief drugs <7 days, 7-14 days, >14 days

In [48]:
## Group by ["patientid", "days_to_covid_diag"], inspect a reasonable response cutoff
df_med_groupby = df_med_dedup.groupby(['patientid', 'days_to_covid_diag'])[['drug_name']].count()

In [50]:
df_med_groupby.head(1000)

drug_name
patientid     days_to_covid_diag           
RAADC3-000002 8                           2
              28                          1
              29                          1
              36                          1
              56                         24
...                                     ...
RAADC3-000456 2                           1
              9                           1
              35                          4
RAADC3-000459 8                           2
              43                          1

[1000 rows x 1 columns]

### Each patient may have multiple visits, and multiple drugs prescribed each visit. 
### Report the smallest "days_to_covid_diag" as earlist med treatment

In [56]:
df_med_earliest_med_response = df_med_dedup.groupby(['patientid'])[['days_to_covid_diag']].min().reset_index()

In [57]:
df_med_earliest_med_response

,patientid,days_to_covid_diag
0,RAADC3-000002,8
1,RAADC3-000005,56
2,RAADC3-000006,6
3,RAADC3-000008,1
4,RAADC3-000011,15
...,...,...
208750,RAADC3-727949,4
208751,RAADC3-727950,3
208752,RAADC3-727951,24
208753,RAADC3-727954,2


In [58]:
df_med_earliest_med_response.columns = [['patientid', 'earliest_med_response_to_covid_diag']]

In [59]:
df_med_earliest_med_response.head(5)

,patientid,earliest_med_response_to_covid_diag
0,RAADC3-000002,8
1,RAADC3-000005,56
2,RAADC3-000006,6
3,RAADC3-000008,1
4,RAADC3-000011,15


In [60]:
df_med_earliest_med_response.shape

(208755, 2)

In [61]:
df_med_earliest_med_response.to_csv("/home/huangz36/med_table_earliest_med_response_by_patientid.txt", sep='\t', \
                                index=False, header=True)

In [62]:
### Now read-in the table of top 100 common drugs, with annotated categories
### Location: /home/huangz36/Top100_drug_with_annotated_category.txt

### Categories:
##### 1. Antibiotic
##### 2. Anxiety
##### 3. Breath
##### 4. Cholesterol
##### 5. Cough
##### 6. diabetes
##### 7. fever_pain
##### 8. heart_blood
##### 9. hormone
##### 10. immune
##### 11. infection
##### 12. Muscle
##### 13. nausea_vomit
##### 14. Nutrition
##### 15. others

In [63]:
df_top_100_drug = pd.read_csv("/home/huangz36/Top100_drug_with_annotated_category.txt", sep='\t', header=0)

In [64]:
df_top_100_drug.head()

,Drug,Category
0,Acetaminophen,fever_pain
1,Aspirin,fever_pain
2,Atorvastatin Calcium,heart_blood
3,Sodium Chloride,others
4,Albuterol Sulfate,breath


In [65]:
df_med_dedup.head()

,patientid,days_to_covid_diag,drug_name,ndc
0,RAADC3-504477,21,Mupirocin,68462018022
1,RAADC3-241668,15,Glucose Meter Test Control Strips,53885024450
2,RAADC3-125094,55,Ergocalciferol,69452015120
3,RAADC3-349386,15,Vitamin B Complex and Vitamin C,60258016001
4,RAADC3-556667,21,Tiagabine Hydrochloride,00093503156


In [67]:
## Focus on the top 100 commonly seen drugs in this dataset, drug the relatively less frequently used drugs
## Map these drugs to categories

top100_drug_list = df_top_100_drug['Drug'].to_list()
len(top100_drug_list)

100

In [69]:
df_med_dedup_top100_drugs = df_med_dedup[df_med_dedup['drug_name'].isin(top100_drug_list)]

print(df_med_dedup.shape)
print(df_med_dedup_top100_drugs.shape)

(4473741, 4)
(2549551, 4)


In [70]:
df_med_dedup_top100_drugs_and_cat = df_med_dedup_top100_drugs\
.merge(df_top_100_drug, how='left', left_on='drug_name', right_on='Drug')

In [71]:
df_med_dedup_top100_drugs_and_cat.shape

(2549551, 6)

In [72]:
df_med_dedup_top100_drugs_and_cat.head()

,patientid,days_to_covid_diag,drug_name,ndc,Drug,Category
0,RAADC3-241668,15,Glucose Meter Test Control Strips,53885024450,Glucose Meter Test Control Strips,Nutrition
1,RAADC3-125094,55,Ergocalciferol,69452015120,Ergocalciferol,Nutrition
2,RAADC3-392540,17,Multivitamin and Minerals,80681000300,Multivitamin and Minerals,Nutrition
3,RAADC3-068526,1,Ibuprofen,68094060062,Ibuprofen,fever_pain
4,RAADC3-659802,58,Gabapentin,00378542705,Gabapentin,seizure


In [83]:
## Aggregate and pivot on categories

df_med_top100_drug_by_category_agg = df_med_dedup_top100_drugs_and_cat.\
groupby(['patientid', 'Category'])[['drug_name']].count().reset_index()
df_med_top100_drug_by_category_agg.columns = ['patientid', 'Category', 'Drug_count']

In [84]:
df_med_top100_drug_by_category_agg.head()

,patientid,Category,Drug_count
0,RAADC3-000002,Anxiety,11
1,RAADC3-000002,Diabetes,6
2,RAADC3-000002,Infection,3
3,RAADC3-000002,Nutrition,18
4,RAADC3-000002,diabetes,1


In [91]:
df_med_top100_drug_by_category_agg.shape

(638322, 3)

In [85]:
df_med_top100_drug_pivot_table = df_med_top100_drug_by_category_agg.pivot(index=["patientid"], \
                         columns=["Category"], values=['Drug_count']).fillna(0).reset_index()

In [87]:
df_med_top100_drug_pivot_table.head()

patientid Drug_count                                            \
Category                Antibiotic Anxiety Cholesterol Cough Diabetes Immune   
0         RAADC3-000002        0.0    11.0         0.0   0.0      6.0    0.0   
1         RAADC3-000005        0.0     0.0         0.0   0.0      0.0    0.0   
2         RAADC3-000011        0.0     3.0         0.0   0.0      0.0    0.0   
3         RAADC3-000012        0.0     1.0         0.0   0.0      0.0    0.0   
4         RAADC3-000014        0.0     2.0         0.0   0.0      0.0    2.0   

                                                                     \
Category Infection Muscle Nutrition Skin breath diabetes fever_pain   
0              3.0    0.0      18.0  0.0    0.0      1.0       21.0   
1              0.0    0.0       0.0  0.0    0.0      0.0        0.0   
2              6.0    0.0       0.0  1.0    1.0      0.0        0.0   
3              0.0    0.0       0.0  0.0    0.0      0.0        1.0   
4              0.0    0.0       4.0  2.0    2.0      0.0        5.0   

                                                                  
Category heart_blood hormone nausea_vomit others seizure stomach  
0               18.0     0.0          0.0   11.0     0.0     6.0  
1                1.0     0.0          0.0    0.0     0.0     0.0  
2                5.0     0.0          0.0    1.0     0.0     1.0  
3                0.0     0.0          0.0    0.0     0.0     0.0  
4               12.0     2.0          0.0    1.0     2.0     2.0

In [88]:
new_column_names = []
for (row1, row2) in df_med_top100_drug_pivot_table.columns:
    column_name = str(row1) + "_" + str(row2)
    print(column_name)
    new_column_names.append(column_name)

patientid_
Drug_count_Antibiotic
Drug_count_Anxiety
Drug_count_Cholesterol
Drug_count_Cough
Drug_count_Diabetes
Drug_count_Immune
Drug_count_Infection
Drug_count_Muscle
Drug_count_Nutrition
Drug_count_Skin
Drug_count_breath
Drug_count_diabetes
Drug_count_fever_pain
Drug_count_heart_blood
Drug_count_hormone
Drug_count_nausea_vomit
Drug_count_others
Drug_count_seizure
Drug_count_stomach


In [89]:
df_med_top100_drug_pivot_table.columns = new_column_names

In [90]:
df_med_top100_drug_pivot_table.shape

(182882, 20)

In [92]:
df_med_top100_drug_pivot_table.head()

,patientid_,Drug_count_Antibiotic,Drug_count_Anxiety,Drug_count_Cholesterol,Drug_count_Cough,Drug_count_Diabetes,Drug_count_Immune,Drug_count_Infection,Drug_count_Muscle,Drug_count_Nutrition,Drug_count_Skin,Drug_count_breath,Drug_count_diabetes,Drug_count_fever_pain,Drug_count_heart_blood,Drug_count_hormone,Drug_count_nausea_vomit,Drug_count_others,Drug_count_seizure,Drug_count_stomach
0,RAADC3-000002,0.0,11.0,0.0,0.0,6.0,0.0,3.0,0.0,18.0,0.0,0.0,1.0,21.0,18.0,0.0,0.0,11.0,0.0,6.0
1,RAADC3-000005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,RAADC3-000011,0.0,3.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,1.0,1.0,0.0,0.0,5.0,0.0,0.0,1.0,0.0,1.0
3,RAADC3-000012,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,RAADC3-000014,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,4.0,2.0,2.0,0.0,5.0,12.0,2.0,0.0,1.0,2.0,2.0


In [93]:
## output the table per patient_id, with counts of drugs per category as features
df_med_top100_drug_pivot_table.to_csv("/home/huangz36/med_table_pivot_by_drug_category_and_counts.txt", sep='\t', \
                                index=False, header=True)